In [ ]:
from html.parser import HTMLParser
import requests
import csv
import re


class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.text = ""
        self.in_script = False
        self.in_style = False

    def handle_data(self, data):
        if not self.in_script and not self.in_style:
            self.text += data

    def handle_starttag(self, tag, attrs):
        if tag == "script":
            self.in_script = True
        elif tag == "style":
            self.in_style = True

    def handle_endtag(self, tag):
        if tag == "script":
            self.in_script = False
        elif tag == "style":
            self.in_style = False

    def parse_html(self, html_content):
        self.text = ""  # Initialize self.text for each parsing
        self.feed(html_content)
        # Remove empty lines and preceding left spaces
        text_content_lines = [re.sub(r'\s+', ' ', line.strip()) for line in self.text.splitlines() if line.strip()]
        self.text = "\n".join(text_content_lines)
        # Remove non-UTF characters
        self.text = re.sub(r'[^\x00-\x7F]+', '', self.text)

        return self.text
 
parser = MyHTMLParser()

with open('urls2.csv', 'r') as file:
    
    # Create a CSV reader
    csv_reader = csv.reader(file)

    # Skip the header row if it contains column names
    next(csv_reader)

    # Iterate through each row in the CSV
    for row in csv_reader:
        with open(f'text/{row[1]}.txt', 'w') as output:
            print(row[0])
            # Send an HTTP GET request to the URL
            response =  requests.get(row[0])

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Get the HTML content of the webpage
                html_content = response.text
                parser.parse_html(html_content)
                text_content = parser.text
                
                #write to file
                output.write((text_content) + "\n")
                text_content_lines = ""



In [ ]:
import csv
import os

with open('urls2.csv', 'r') as file:
    reader = csv.reader(file, delimiter=',')
    next(reader)  # skip the header row
    data = list(reader)
    
# Function to split the text into chunks of approximately 1000 words
def split_into_chunks(text, word_limit=1000):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        if len(current_chunk) < word_limit:
            current_chunk.append(word)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

all_data = []
id_counter = 31
for (url, name) in data:
    try:
        with open(os.path.join('text', f"{name}.txt"), 'r', encoding="utf-8") as txt_file:
            raw_text = txt_file.read().replace("\n", " ").replace("\r", " ").strip()  # replacing newlines with spaces
            raw_text = raw_text.replace(",", " ")  # replace commas with spaces
            
            # Split the raw_text into chunks of approximately 1000 words
            text_chunks = split_into_chunks(raw_text)
            for chunk in text_chunks:
                all_data.append((id_counter, url, chunk))
                id_counter += 1
            
    except FileNotFoundError:
        print(f"Warning: File for {name} not found.")

with open('test_all_data.csv', 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['id', 'url', 'raw text'])  # write the header
    for row in all_data:
        writer.writerow(row)

# URLS
https://www.verizon.com/about/our-company/innovation-labs
https://www.verizon.com/about/responsibility/
https://www.verizon.com/business/
https://www.verizon.com/business/products/
https://www.verizon.com/business/shop/products/devices/smartphones
https://www.verizon.com/business/solutions/
https://www.verizon.com/business/solutions/small-business/
https://www.verizon.com/business/why-verizon/
https://www.verizon.com/home/internet/
https://www.verizon.com/plans/
https://www.verizon.com/plans/devices/
https://www.verizon.com/plans/international/
https://www.verizon.com/plans/unlimited/
https://www.verizon.com/prepaid/
https://www.verizon.com/support/account-management/
https://www.verizon.com/support/billing-and-payments/
https://www.verizon.com/support/homepage/
https://www.verizon.com/support/knowledge-base-206960/
https://www.verizon.com/support/services-and-apps/
